In [1]:
import os
import sys
import ifcopenshell as _ifc
import pandas as pd
import openpyxl

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

course_path = os.path.join(project_root, 'Course')
if course_path not in sys.path:
    sys.path.append(course_path)

from _general import mguu_cource_tools

In [3]:
def getting_super_table_by_ifc(ifc_file_path):
    ifc_file = _ifc.open(ifc_file_path)
    ifc_objects = ifc_file.by_type("IfcObject")
    list_temp = list()
    total_props_names = list()
    for ifc_entity in ifc_objects:
        temp_props = mguu_cource_tools.get_object_properties(ifc_entity)
        list_temp.append(temp_props)
        # merge props
        current_names = temp_props.keys()
        for key in current_names:
            if key not in total_props_names:
                total_props_names.append(key)
    out_data = dict()
    for key in total_props_names:
        out_data[key] = []
    for entity_data in list_temp:
        for pr in total_props_names:
            if pr in entity_data.keys():
                out_data[pr].append(entity_data[pr])
            else:
                out_data[pr].append(None)
    df = pd.DataFrame(data=out_data, columns=total_props_names)
    return df

In [8]:
example_file_path = os.path.join(project_root, 'DataExamples', 'rme_advanced_sample_project.ifc')

In [9]:
ifc_objects_data = mguu_cource_tools.getting_super_table_by_ifc(example_file_path)

1

In [12]:
def get_object_properties(ifc_entity):
    temp_props = {"IfcClass": ifc_entity.is_a(), "GlobalId": ifc_entity.GlobalId}
    ifc_props_root = ifc_entity.IsDefinedBy
    for props_group in ifc_props_root:
        if props_group.is_a("IfcRelDefinesByProperties"):
            props_definition = props_group.RelatingPropertyDefinition

            def get_prop_data(props_array):
                for ps in props_array:
                    p_name = ps.Name

                    # print(p_name)

                    def check_prop(check_value):
                        if check_value is not None:
                            if isinstance(check_value, (int, float, bool, str)):
                                temp_props[p_name] = check_value
                            else:
                                d_new = check_value.get_info()
                                for k2, v2 in d_new.items():
                                    check_prop(v2)
                        pass

                    for k1, v1 in ps.get_info().items():
                        check_prop(v1)
                pass

            # property sets
            if props_definition.is_a("IfcPropertySet"):
                get_prop_data(props_definition.HasProperties)
            # qto
            if props_definition.is_a("IfcElementQuantity"):
                get_prop_data(props_definition.Quantities)
    return temp_props

In [11]:
def getting_summa_of_prop_by_category(ifc_file, needing_category_names, needing_prop_sum_names):
    ifc_objects = ifc_file.by_type("IfcObject")
    ifc_objects_new = list((filter(lambda x: x.is_a() in needing_category_names, ifc_objects)))
    out_dict = {"Object name" : [], "Total value": []}
    for ifc_entity in ifc_objects_new:
        #print(ifc_entity.get_info())
        entity_name = ifc_entity.ObjectType
        temp_props = get_object_properties(ifc_entity)
        # calc_values
        for k, v in temp_props.items():
            if k in needing_prop_sum_names:
                if entity_name not in out_dict["Object name"]:
                    out_dict["Object name"].append(entity_name)
                    out_dict["Total value"].append(v)
                else:
                    index_that_name = out_dict["Object name"].index(entity_name)
                    out_dict["Total value"][index_that_name] += v
                break
    #print(str(out_dict))
    df = pd.DataFrame(data=out_dict, columns=out_dict.keys())
    return df

In [10]:
ifc_file = _ifc.open(example_file_path)
ifc_file

In [13]:
ifc_summa_info = getting_summa_of_prop_by_category(ifc_file, ["IfcWall", "IfcWallStandardCase"], ["Volumen", "NetVolume", "Volume"])
ifc_summa_info

,Object name,Total value
0,Basiswand:STB 30.0,88.571908
1,Basiswand:STB 20.0,173.773261
2,Basiswand:STB 30.0 Rot,16.828197
3,Basiswand:MW 11.5,141.296721
4,Basiswand:MW 17.5,67.460730
5,Basiswand:Lamelle 11.5,74.820251
6,Basiswand:WC Trennwand 5.0,1.610856
7,Basiswand:STB 25.0 WD 12.0,7.326000


In [16]:
ifc_summa_info.to_excel('ifc_summa.xlsx')